In [2]:
import numpy as np
import pandas as pd
import os
#from osgeo import gdal
import geopandas as gpd
#import rasterio
import osmnx as ox
import rioxarray
import pyproj
import shapely.geometry as sg
import networkx as nx
import momepy

In [3]:
# Test data
test_path = "C:/Users/ygrin/Documents/Studie - MSc ADS/Utrecht University/Block 4 - Thesis/TestData/"
single_point = gpd.read_file(test_path + "Test_single_home_location.gpkg")
multi_point = gpd.read_file(test_path + "Test_multiple_home_locations.gpkg")
#polygon = gpd.read_file(test_path + "TestArea.gpkg")
ndvi = rioxarray.open_rasterio(test_path + "NDVI_data_test.tif")
land_cover = rioxarray.open_rasterio(test_path + "Landcover_data_test.tif")
#network = gpd.read_file(test_path + "test_network.gpkg", layer='edges')
network = gpd.read_file(test_path + "test_network_shp.shp")
polygon = gpd.read_file(test_path + "test_polygon.gpkg")
canopy_single = gpd.read_file(test_path + "Canopy_single_home_500m.gpkg")

single_point_geographic = single_point.to_crs('epsg:4326')

# Greenness Accessibility -- Shortest Network Distance to Park

In [4]:
def get_shortest_distance_park(point_of_interest_file, park_vector_file=None, buffer_dist=None, network_file=None, network_type=None,
                               output_dir=os.getcwd()):
    # Read and process user input, check conditions
    poi = gpd.read_file(point_of_interest_file)
    if all(poi['geometry'].geom_type == 'Point') or all(poi['geometry'].geom_type == 'Polygon'):
        geom_type = poi.iloc[0]['geometry'].geom_type
    else:
        raise ValueError("Please make sure all geometries are of 'Point' type or all geometries are of 'Polygon' type and re-run the function")

    if not poi.crs.is_projected:
        raise ValueError("The CRS of the PoI dataset is currently geographic, please transform it into a local projected CRS and re-run the function")
    else:
        epsg = poi.crs.to_epsg()

    if geom_type == "Polygon":
        print("Changing geometry type to Point by computing polygon centroids so that network distance can be retrieved...")
        poi['geometry'] = poi['geometry'].centroid
        print("Done \n")

    if "id" in poi.columns:
        if poi['id'].isnull().values.any():
            poi['id'] = poi['id'].fillna(pd.Series(range(1, len(poi) + 1))).astype(int)
    else:
        poi['id'] = pd.Series(range(1, len(poi) + 1)).astype(int)

    if park_vector_file is not None:
        park_src = gpd.read_file(park_vector_file)
        if not park_src.crs.to_epsg() == epsg:
            print("Adjusting CRS of Park file to match with Point of Interest CRS...")
            park_src.to_crs(f'EPSG:{epsg}', inplace=True)
            print("Done \n")
    else:
        epsg_transformer = pyproj.Transformer.from_crs(f"epsg:{epsg}", "epsg:4326")
        park_tags = {'leisure': 'park', 'boundary': 'national_park', 'landuse': 'recreation_ground'}
        park_src = gpd.GeoDataFrame()
        for geom in multi_point['geometry']:
            latlon = epsg_transformer.transform(geom.x, geom.y)
            park_geom = ox.geometries_from_point(latlon, tags=park_tags, dist=500)
            park_src = gpd.GeoDataFrame(pd.concat([park_src, park_geom], ignore_index=True), crs=park_geom.crs)
        park_src.to_crs(f"EPSG:{epsg}")
        

    return poi

In [18]:
epsg_transformer = pyproj.Transformer.from_crs("epsg:27700", "epsg:4326")
park_tags = {'leisure': 'park', 'boundary': 'national_park', 'landuse': 'recreation_ground'}
park_src = gpd.GeoDataFrame()
for geom in multi_point['geometry']:
    latlon = epsg_transformer.transform(geom.x, geom.y)
    park_geom = ox.geometries_from_point(latlon, tags=park_tags, dist=500)
    park_src = gpd.GeoDataFrame(pd.concat([park_src, park_geom], ignore_index=True), crs=park_geom.crs)
park_src

,nodes,leisure,name,wikidata,wikipedia,geometry,surface,operator,website
0,"[9466933939, 371536259, 371536261, 371536262, ...",park,Highfield Country Park,Q15223939,en:Highfield Country Park,"POLYGON ((-2.17167 53.43757, -2.17359 53.43775...",NaN,NaN,NaN
1,"[1746663515, 1746663535, 1746663518, 174666352...",park,NaN,NaN,NaN,"POLYGON ((-2.17657 53.43026, -2.17736 53.43104...",NaN,NaN,NaN
2,"[7928740934, 7928740935, 7928744009, 792874093...",park,NaN,NaN,NaN,"POLYGON ((-2.16871 53.42920, -2.16756 53.42893...",grass,NaN,NaN
3,"[21628679, 21628680, 21628681, 21628682, 21628...",park,Kingswood Park,NaN,NaN,"POLYGON ((-2.21323 53.43852, -2.21343 53.43872...",NaN,NaN,NaN
4,"[6272922724, 6272922725, 6272922726, 627292272...",park,Sunnybrow Park,NaN,NaN,"POLYGON ((-2.17556 53.45971, -2.17391 53.46016...",NaN,Manchester City Council,https://www.manchester.gov.uk/directory_record...
